In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

在GCP上的E2E ML：MLOps阶段6：serving：使用Vertex AI Online Prediction为AutoML表格模型入门

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_automl_tabular_model_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_automl_tabular_model_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage6/get_started_with_automl_tabular_model_online.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

概述

本教程演示如何使用Vertex AI SDK创建表格二元分类模型，并使用AutoML模型进行在线预测和解释。

目标

在本教程中，您将从Python脚本中创建一个AutoML表格二元分类模型，然后使用Vertex AI SDK进行在线预测。您还可以选择使用`gcloud`命令行工具或通过Cloud Console在线创建和部署模型。

本教程使用以下谷歌云ML服务和资源：

- `Vertex AutoML`
- `Vertex AI Online Prediction`
- `Vertex Explainable AI`
- `Vertex AI Datasets`
- `Vertex AI Models`
- `Vertex AI Endpoints`

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练一个`AutoML`表格模型。
- 将模型部署到一个`Endpoint`资源。
- 进行在线预测。
- 使用解释进行在线预测。

### 数据集

本教程使用的数据集是[银行营销](https://pantheon.corp.google.com/storage/browser/_details/cloud-ml-tables-data/bank-marketing.csv)。这个数据集不需要任何特征工程。您在本教程中使用的数据集版本存储在公共云存储桶中。

### 费用

本教程使用 Google Cloud 的收费组件：

- Vertex AI
- 云存储

了解关于[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)，[云存储价格](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成费用估算。

### 设置本地开发环境

如果您正在使用Colab或Vertex AI Workbench笔记本，您的环境已经满足运行本笔记本的所有要求。您可以跳过这一步。

否则，请确保您的环境满足本笔记本的要求。您需要以下内容：

- 云存储SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

云存储指南[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了一套简洁的说明：

1. [安装和初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，在终端shell中运行`pip3 install jupyter`。

5. 要启动Jupyter，在终端shell中运行`jupyter notebook`。

6. 在Jupyter笔记本仪表板中打开此笔记本。

## 安装

安装以下包以执行这个笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade {USER_FLAG} -q google-cloud-aiplatform \
                                        tabulate

### 重新启动内核

安装了额外的包之后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

##开始之前

###GPU运行时

此教程不需要GPU运行时。

###设置您的Google Cloud项目

**无论您使用什么笔记本环境，下面的步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。您第一次创建帐户时，您会获得$300的免费信用额度，可用于计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API：Vertex AI API、Compute Engine API和Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK在此笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter会将以`!`为前缀的行作为shell命令运行，并会替换以`$`为前缀的Python变量。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改“REGION”变量，该变量用于本笔记本的其余部分操作。以下是Vertex AI支持的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法使用多区域存储桶来进行Vertex AI的训练。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

UUID

如果您正在进行实时教程会话，您可能正在使用共享的测试帐户或项目。为了避免用户在创建的资源之间发生名称冲突，您为每个实例会话创建一个uuid，并将其附加到您在本教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 认证您的Google Cloud帐户

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经经过身份验证。

**如果您正在使用Colab**，请运行下面的单元格，并按照提示进行身份验证，通过OAuth。

**否则**，请按照以下步骤操作：

在Cloud Console中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中输入名称，然后点击**创建**。

在**授予此服务帐户访问项目权限**部分，点击角色下拉列表。在过滤框中输入"Vertex"，然后选择**Vertex管理员**。在过滤框中输入"Storage Object Admin"，然后选择**存储对象管理员**。

点击创建。包含您的密钥的JSON文件将下载到您的本地环境。

在下面的单元格中将您的服务帐户密钥的路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的是哪种笔记本环境，都需要执行以下步骤。**

当您初始化用于 Python 的 Vertex AI SDK 时，需要指定一个云存储暂存桶。这个暂存桶是您的数据集和模型资源在会话之间保留的地方。

请在下方设置您的云存储桶的名称。存储桶的名称必须在所有谷歌云项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = "gs://" + BUCKET_NAME

只有在您的储存桶不存在时才能执行以下单元格以创建您的云存储储存桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

###设定变量

接下来，设定一些在本教程中使用的变量。
###导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform
from tabulate import tabulate

初始化Vertex AI SDK适用于Python

为您的项目和相应的存储桶初始化Vertex AI SDK适用于Python。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### 设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量 `DEPLOY_GPU/DEPLOY_NGPU` 设置为使用支持 GPU 的容器镜像，以及分配给虚拟机实例（VM）的 GPU 数量。例如，要使用一个带有 4 个 Nvidia Telsa K80 GPU 的容器镜像分配到每个 VM，您可以指定：

(aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，指定`(None, None)`来使用一个在 CPU 上运行的容器镜像。

了解有关[您所在区域的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

了解更多关于[机器类型的 GPU 兼容性](https://cloud.google.com/vertex-ai/docs/training/configure-compute#gpu-compatibility-table)。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 将变量 `DEPLOY_COMPUTE` 设置为配置用于预测的VM所需的计算资源。
 - `机器类型`
     - `n1-standard`: 每个vCPU有3.75GB内存。
     - `n1-highmem`: 每个vCPU有6.5GB内存。
     - `n1-highcpu`: 每个vCPU有0.9GB内存。
 - `vCPUs`: \[2, 4, 8, 16, 32, 64, 96\]中的数量

*注意：您也可以在训练和部署时使用n2和e2机器类型，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", DEPLOY_COMPUTE)

现在您可以开始创建自己的AutoML表格二分类模型。

云存储训练数据的位置。

现在将变量 `IMPORT_FILE` 设置为云存储中 CSV 索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-ml-tables-data/bank-marketing.csv"

#### 快速查看您的数据

本教程使用存储在公共云存储桶中的一个版本的银行营销数据集，使用CSV索引文件。

首先快速查看数据。通过计算CSV索引文件中的行数（`wc -l`）来计算示例数，然后查看前几行。

您还需要知道训练时标签列的标题名称，这个名称保存为`label_column`。对于这个数据集，它是CSV文件中的最后一列。

In [ ]:
count = ! gsutil cat $IMPORT_FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $IMPORT_FILE | head

heading = ! gsutil cat $IMPORT_FILE | head -n1
label_column = str(heading).split(",")[-1].split("'")[0]
print("Label Column Name", label_column)
if label_column is None:
    raise Exception("label column missing")

### 创建数据集

接下来，使用`TabularDataset`类的`create`方法创建`Dataset`资源，需要以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `gcs_source`：一个或多个数据集索引文件的列表，用于将数据项导入`Dataset`资源中。
- `bq_source`：或者，从BigQuery表中导入数据项到`Dataset`资源中。

此操作可能需要几分钟。

In [ ]:
dataset = aiplatform.TabularDataset.create(
    display_name="Bank Marketing_" + UUID, gcs_source=[IMPORT_FILE]
)

print(dataset.resource_name)

### 创建并运行训练流水线

为了训练一个AutoML模型，您需要执行两个步骤：1）创建一个训练流水线，2）运行流水线。

#### 创建训练流水线

使用`AutoMLTabularTrainingJob`类创建一个AutoML训练流水线，指定以下参数：

- `display_name`：用于`TrainingJob`资源的可读名称。
- `optimization_prediction_type`：要为模型训练的任务类型。
  - `classification`：一个表格分类模型。
  - `regression`：一个表格回归模型。
- `column_transformations`：（可选）要应用于输入列的变换。
- `optimization_objective`：要最小化或最大化的优化目标。
  - 二元分类：
    - `minimize-log-loss`
    - `maximize-au-roc`
    - `maximize-au-prc`
    - `maximize-precision-at-recall`
    - `maximize-recall-at-precision`
  - 多类别分类：
    - `minimize-log-loss`
  - 回归：
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

实例化的对象是训练流水线的DAG（有向无环图）。

In [ ]:
dag = aiplatform.AutoMLTabularTrainingJob(
    display_name="bank_" + UUID,
    optimization_prediction_type="classification",
    optimization_objective="minimize-log-loss",
)

print(dag)

#### 运行训练管道

接下来，您可以运行DAG来启动训练作业，通过调用`run`方法，并传入以下参数：

- `dataset`：要训练模型的`Dataset`资源。
- `model_display_name`：训练模型的可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。
- `target_column`：作为标签进行训练的列名。
- `budget_milli_node_hours`：（可选）以毫小时为单位指定的最大训练时间（1000 = 小时）。
- `disable_early_stopping`：如果设置为`True`，则在服务认为无法进一步改善模型目标测量时，训练可能会提前完成预算使用。

完成`run`方法后将返回`Model`资源。

执行训练管道可能需要高达8小时。

*注意：*与训练定制模型不同，经过训练的AutoML模型的相应服务二进制文件是预定义的。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="bank_" + UUID,
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
    target_column=label_column,
)

创建 `Endpoint` 资源

您可以使用 `Endpoint.create()` 方法创建 `Endpoint` 资源。至少，您需要指定端点的显示名称。可选地，您可以指定项目和位置（区域）；否则，设置会继承您使用 `init()` 方法初始化 Vertex AI SDK 时设置的值。

在此示例中，指定了以下参数：

- `display_name`：`Endpoint` 资源的可读名称。
- `project`：您的项目 ID。
- `location`：您的区域。
- `labels`：（可选）`Endpoint` 的用户定义元数据，以键/值对的形式。

此方法返回一个 `Endpoint` 对象。

了解更多关于 [Vertex AI Endpoints](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)。

In [ ]:
endpoint = aiplatform.Endpoint.create(
    display_name="bank_" + UUID,
    project=PROJECT_ID,
    location=REGION,
    labels={"your_key": "your_value"},
)

print(endpoint)

将`Model`资源部署到`Endpoint`资源。

您可以将一个或多个`Vertex AI Model`资源实例部署到同一个端点。每个部署的`Vertex AI Model`资源将拥有自己的用于服务二进制模型的部署容器。

*注意：* 对于本示例，您的`AutoML`模型的部署容器是预定义的。

### 扩展

`Vertex AI Endpoint`资源支持三种类型的扩展：

- 无扩展：服务二进制部署到单个VM实例。
- 手动扩展：服务二进制部署到固定数量的多个VM实例。
- 自动扩展：根据负载的不同，服务二进制被部署到的VM实例数量会有所变化。

对于AutoML模型，只支持无扩展和手动扩展。

### 手动扩展

在下一个例子中，您会将`Vertex AI Model`资源部署到一个`Vertex AI Endpoint`资源中进行手动扩展——固定数量（大于1）的虚拟机实例。换句话说，当模型部署时，固定数量的虚拟机实例被预配并保持预配直到模型被取消部署。

在这个例子中，您将使用最少数量的指定参数部署模型，如下所示：

- `model`：要部署的`Model`资源。
- `machine_type`：每个虚拟机实例的机器类型。
- `deployed_model_displayed_name`：部署模型实例的可读名称。
- `min_replica_count`：要预配的最小虚拟机实例（节点）数量。
- `max_replica_count`：要预配的最大虚拟机实例（节点）数量。
- `accelerator_type`：每个虚拟机实例的硬件加速器类型，如果有的话。
- `accelerator_count`：每个虚拟机实例的硬件加速器数量，如果有的话。

对于手动扩展，固定数量的虚拟机实例在模型部署期间被预配。

*注意：* 对于手动扩展，节点的最小数和最大数设置为相同值。

In [ ]:
MIN_NODES = 2
MAX_NODES = 2

if DEPLOY_NGPU:
    response = endpoint.deploy(
        model=model,
        deployed_model_display_name="bank_" + UUID,
        machine_type=DEPLOY_COMPUTE,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
        accelerator_type=DEPLOY_GPU.name,
        accelerator_count=DEPLOY_NGPU,
    )
else:
    response = endpoint.deploy(
        model=model,
        deployed_model_display_name="bank_" + UUID,
        machine_type=DEPLOY_COMPUTE,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )

发送一个在线预测请求

向您部署的模型发送一个在线预测。

制作测试项目

您将使用合成数据作为测试数据项。

In [ ]:
INSTANCE = {
    "Age": "58",
    "Job": "managment",
    "MaritalStatus": "married",
    "Education": "teritary",
    "Default": "no",
    "Balance": "2143",
    "Housing": "yes",
    "Loan": "no",
    "Contact": "unknown",
    "Day": "5",
    "Month": "may",
    "Duration": "261",
    "Campaign": "1",
    "PDays": "-1",
    "Previous": "0",
    "POutcome": "unknown",
}

### 进行预测

现在您的`Model`资源已部署到一个`Endpoint`资源上，您可以通过向`Endpoint`资源发送预测请求来进行在线预测。

#### 请求

每个实例的格式为：

    [feature_list]

由于`predict()`方法可以接受多个项目（实例），请将您的单个测试项作为一个测试项列表发送。

#### 响应

从`predict()`调用的响应是一个Python字典，包含以下条目：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `displayNames`：每个类标签的类名。
- `confidences`：每个类标签的预测置信度，介于0和1之间。
- `deployed_model_id`：执行预测的已部署`Model`资源的Vertex AI标识符。

In [ ]:
instances_list = [INSTANCE]

prediction = endpoint.predict(instances_list)
print(prediction)

发送一个带有可解释性的在线预测请求

向部署的模型发送一个带有可解释性的在线预测请求。在这种方法中，预测的响应将包括一个关于特征如何 contributed to the explanation的解释。

现在，您的“模型”资源已部署到“端点”资源，可以通过向“端点”资源发送预测请求来进行在线解释。

#### 请求

每个实例的格式为：

    [feature_list]

由于explain()方法可以接受多个项目（实例），请将您的单个测试项目作为一个测试项目列表发送。

#### 响应

从explain()调用中得到的响应是一个包含以下条目的Python字典：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `displayNames`：每个类别标签的类别名称。
- `confidences`：对于分类任务，每个类别标签的预测置信度，介于0和1之间。
- `values`：对于回归任务，预测的值。
- `deployed_model_id`：执行预测的部署的“模型”资源的Vertex AI标识符。
- `explanations`：特征归因。

In [ ]:
instances_list = [INSTANCE]

prediction = endpoint.explain(instances_list)
print(prediction)

### 理解解释的回应

首先，您需要查看您的模型预测结果并将其与实际值进行比较。

In [ ]:
import numpy as np

try:
    label = np.argmax(prediction[0][0]["scores"])
    cls = prediction[0][0]["classes"][label]
    print("Predicted Value:", cls, prediction[0][0]["scores"][label])
except:
    pass

### 检查特征归因

接下来，您将查看此特定示例的特征归因。积极的归因值意味着特定特征将您的模型预测值上升了该量，反之负值意味着特征将您的模型预测值下降了该量。

归因值的总和等于基准平均值和预测值之间的差值，例如，`baseline_average_value - predicted_value)。

In [ ]:
feature_names = [
    "Age",
    "Job",
    "MaritalStatus",
    "Education",
    "Default",
    "Balance",
    "Housing",
    "Loan",
    "Contact",
    "Day",
    "Month",
    "Duration",
    "Campaign",
    "PDays",
    "Previous",
    "POutcome",
]
attributions = prediction.explanations[0].attributions[0].feature_attributions

rows = []
for i, val in enumerate(feature_names):
    rows.append([val, INSTANCE[val], attributions[val]])
print(tabulate(rows, headers=["Feature name", "Feature value", "Attribution value"]))

### 检查您的解释和基线

为了更好地理解您获得的特征归因，您应该将它们与您模型的基线进行比较。在大多数情况下，您的归因值之和加上基线应该非常接近每个输入的模型预测值。另外，请注意，对于回归模型，从 AI 解释返回的`baseline_score`将对发送到您的模型的每个示例都是相同的。对于分类模型，每个类别将有自己的基线。

在本节中，您将向您的模型发送 10 个测试示例进行预测，以便将特征归因与基线进行比较。然后，您将通过`sanity_check_explanations`方法对每个测试示例的归因进行一次逻辑检查。

In [ ]:
import random

# Prepare 10 test examples to your model for prediction using a random distribution to generate
# test instances
instances = []
for i in range(10):
    age = str(random.randint(25, 65))
    job = ["management", "technican", "unknown"][random.randint(0, 2)]
    status = ["single", "married", "divorced"][random.randint(0, 2)]
    ed = ["primary", "secondary", "tertiary"][random.randint(0, 2)]
    df = ["yes", "no"][random.randint(0, 1)]
    bal = str(random.randint(0, 4000))
    housing = ["yes", "no"][random.randint(0, 1)]
    loan = ["yes", "no"][random.randint(0, 1)]
    contact = "unknown"
    day = str(random.randint(1, 28))
    month = "may"
    dur = str(random.randint(1, 360))
    camp = str(random.randint(0, 1))
    pdays = "-1"
    previous = "0"
    poutcome = "unknown"
    instances.append(
        {
            "Age": age,
            "Job": job,
            "MaritalStatus": status,
            "Education": ed,
            "Default": df,
            "Balance": bal,
            "Housing": housing,
            "Loan": loan,
            "Contact": contact,
            "Day": day,
            "Month": month,
            "Duration": dur,
            "Campaign": camp,
            "PDays": pdays,
            "Previous": previous,
            "POutcome": poutcome,
        }
    )

response = endpoint.explain(instances)

#### 健康检查

在下面的功能中，您对解释进行了健康检查。

In [ ]:
import numpy as np


def sanity_check_explanations(
    explanation, prediction, mean_tgt_value=None, variance_tgt_value=None
):
    passed_test = 0
    total_test = 1
    # `attributions` is a dict where keys are the feature names
    # and values are the feature attributions for each feature
    baseline_score = explanation.attributions[0].baseline_output_value
    print("baseline:", baseline_score)

    # Sanity check 1
    # The prediction at the input is equal to that at the baseline.
    #  Please use a different baseline. Some suggestions are: random input, training
    #  set mean.
    if abs(prediction - baseline_score) <= 0.05:
        print("Warning: example score and baseline score are too close.")
        print("You might not get attributions.")
    else:
        passed_test += 1
        print("Sanity Check 1: Passed")

    print(passed_test, " out of ", total_test, " sanity checks passed.")


i = 0
for explanation in response.explanations:
    try:
        prediction = np.max(response.predictions[i]["scores"])
    except TypeError:
        prediction = np.max(response.predictions[i])
    sanity_check_explanations(explanation, prediction)
    i += 1

取消部署模型

当完成预测后，您可以从`Vertex AI Endpoint`资源中取消部署模型。这将清除所有计算资源并停止已部署模型的计费。

In [ ]:
endpoint.undeploy_all()

删除模型

接下来，您可以使用`delete()`方法删除`Vertex AI Model`资源。

In [ ]:
model.delete()

删除数据集

接下来，您可以使用 `delete()` 方法删除 `Vertex AI Dataset` 资源。

In [ ]:
dataset.delete()

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}